In [3]:
import uuid

<h1> Trimming file </h1> 

In [2]:
#!pip install ffmpeg-python
import ffmpeg
import subprocess
import shlex
import os

In [8]:
def trim_audio(audio_path,genre_name,dest,duration):
    dest_path = r"'%s'" %os.path.join(dest,genre_name+"%09d.wav")
    #print(dest_path)
    command = "ffmpeg -i " + audio_path+ " -f segment -segment_time "+str(duration) + " -c copy " + dest_path
    print(command)
    command = shlex.split(command)
    return subprocess.run(command)
    
    
def trim_by_genre(genre_path,dest,duration):
    genre_name = os.path.basename(genre_path)
    genre_name = "_".join(genre_name.split())
    dest_path = os.path.join(dest,genre_name)
    if not os.path.exists(dest_path):
        os.mkdir(dest_path)
    print(genre_name)
    names = os.listdir(genre_path)
    for i,name in enumerate(names):
        idx = uuid.uuid4().hex
        os.rename(os.path.join(genre_path,name),os.path.join(genre_path,str(i)))
        used_path = "'" +os.path.join(genre_path,str(i)) + "'"
        trim_audio(used_path,idx,dest_path,duration)
        print(name+ ":done!")
    print("-"*5 + genre_name + " Done!" +"-"*5)
    

genre_path = r"C:\Users\nguyenducanh\Desktop\Ballad music" #Folder chua data keo ve
dest = r"C:\Users\nguyenducanh\Desktop\current_proj\DL\final_data_genre" #Folder chua anh thanh da trim
duration = 30
trim_by_genre(genre_path,dest,duration)

Ballad_music
ffmpeg -i 'C:\Users\nguyenducanh\Desktop\Ballad music\0' -f segment -segment_time 30 -c copy 'C:\Users\nguyenducanh\Desktop\current_proj\DL\final_data_genre\Ballad_music\63c493910b204fdb92dca14355dcf209%09d.wav'
90's Acoustic - 90's Music Hits - Best Songs Of The 1990s.mp3:done!
ffmpeg -i 'C:\Users\nguyenducanh\Desktop\Ballad music\1' -f segment -segment_time 30 -c copy 'C:\Users\nguyenducanh\Desktop\current_proj\DL\final_data_genre\Ballad_music\e5a0596ce7184be78c5fb561a9a93d60%09d.wav'
Acoustic 90s 2000s - The Best Acoustic Covers Of Popular Songs 90s 2000s.mp3:done!
ffmpeg -i 'C:\Users\nguyenducanh\Desktop\Ballad music\2' -f segment -segment_time 30 -c copy 'C:\Users\nguyenducanh\Desktop\current_proj\DL\final_data_genre\Ballad_music\0f934d213b32442094896f0dcc4f564f%09d.wav'
Acoustic Ballads - Best Ballads Love Songs Playlist.mp3:done!
ffmpeg -i 'C:\Users\nguyenducanh\Desktop\Ballad music\3' -f segment -segment_time 30 -c copy 'C:\Users\nguyenducanh\Desktop\current_proj\D

Top Ballad Acoustic Love Songs Playlist - English Guitar Acoustic Cover Of Popular Songs Of All Time.mp3:done!
ffmpeg -i 'C:\Users\nguyenducanh\Desktop\Ballad music\28' -f segment -segment_time 30 -c copy 'C:\Users\nguyenducanh\Desktop\current_proj\DL\final_data_genre\Ballad_music\61f3c0e7d6114b869ed7fe0398d5290c%09d.wav'
TOP HITS NHỮNG BẢN NHẠC TRẺ BALLAD HAY NHẤT 2021 - HẸN YÊU FT. NÀNG THƠ.mp3:done!
-----Ballad_music Done!-----


<h1> Extracting File </h1>

In [3]:
import librosa
import numpy as np
import pandas as  pd
import json,pickle
import os

In [6]:
def avg_zero_crossing(array,sr):
    n = len(array)
    s = 0
    for ele in array:
        if ele:
            s += 1
    return s/(n/sr)

def feature_extract(file):
    try:
        """
        Define function that takes in a file an returns features in an array
        """
    
        #get wave representation
        y, sr = librosa.load(file)
        
        #determine if instruemnt is harmonic or percussive by comparing means
        """y_harmonic, y_percussive = librosa.effects.hpss(y)
        if np.mean(y_harmonic)>np.mean(y_percussive):
            harmonic=1
        else:
            harmonic=0"""
        
        #Mel-frequency cepstral coefficients (MFCCs)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        #temporal averaging
        #mfcc=np.mean(mfcc,axis=1)
    
        #get the mel-scaled spectrogram
        spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=64,fmax=8000)  
        #temporally average spectrogram
        #spectrogram = np.mean(spectrogram, axis = 1)
    
        """#compute chroma energy
        chroma = librosa.feature.chroma_cens(y=y, sr=sr)
        #temporally average chroma
        chroma = np.mean(chroma, axis = 1)
    
        #compute spectral contrast
        contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
        contrast = np.mean(contrast, axis= 1)"""
        """
        #avg_zero_crossing
        zero_crossing_feature = avg_zero_crossing(librosa.zero_crossings(y),sr)"""
        
        #return [harmonic,zero_crossing_feature, mfcc, spectrogram, chroma, contrast]
        return [mfcc, spectrogram]
    except:
        print("hey dumbass")
        pass



In [13]:
path = r'C:\Users\nguyenducanh\Desktop\current_proj\DL\final_data_genre\Ballad_music'#Folder chua file am thanh da trim
dest = r'C:\Users\nguyenducanh\Desktop\current_proj\DL\features'#Folder luu file pickle 
names = os.listdir(path)
genre = os.path.basename(path)
re = []
for ele in names:
    #print(ele)
    tmp = feature_extract(os.path.join(path,ele))
    tmp.append(genre)
    re.append(tmp)
d1 = [ele[0] for ele in re]
d2 = [ele[1] for ele in re]
df = pd.DataFrame({"mfcc":d1, "mel-spectrogram":d2})

saved_folder = os.path.join(dest,genre)
os.mkdir(saved_folder)
idx = int(df.shape[0]/10)
start_pos = 0
end_pos = idx
for i in range(10):
    if i < 9:
        tmp = df[:][start_pos:end_pos]
    else:
        tmp = df[:][start_pos:]
        
    tmp.to_pickle(os.path.join(saved_folder,genre + '_feature_bin' + str(i)))
    start_pos = end_pos
    end_pos += idx
        


C:\ProgramData\Anaconda3\lib\site-packages\librosa\core\audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
C:\ProgramData\Anaconda3\lib\site-packages\librosa\core\audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [7]:
def extract_and_save(path,dest = None):
    names = os.listdir(path)
    genre = os.path.basename(path)
    re = []
    for ele in names:
        #print(ele)
        tmp = feature_extract(os.path.join(path,ele))
        tmp.append(genre)
        re.append(tmp)
    d1 = [ele[0] for ele in re]
    d2 = [ele[1] for ele in re]
    df = pd.DataFrame({"mfcc":d1, "mel-spectrogram":d2})

    saved_folder = os.path.join(dest,genre)
    os.mkdir(saved_folder)
    idx = int(df.shape[0]/10)
    start_pos = 0
    end_pos = idx
    for i in range(10):
        if i < 9:
            tmp = df[:][start_pos:end_pos]
        else:
            tmp = df[:][start_pos:]

        tmp.to_pickle(os.path.join(saved_folder,genre + '_feature_bin' + str(i)))
        start_pos = end_pos
        end_pos += idx
    print("-"*5 + "Done" + "-"*5) 

In [8]:
folder = r'C:\Users\nguyenducanh\Desktop\current_proj\DL\final_data_genre'
names = os.listdir(folder)
for ele in names:
    extract_and_save(os.path.join(folder,ele),dest = r'C:\Users\nguyenducanh\Desktop\current_proj\DL\features')

C:\ProgramData\Anaconda3\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=2048 is too small for input signal of length=512
  warnings.warn(


-----Done-----
-----Done-----
-----Done-----
-----Done-----
